# Case Study of the'TinyStories' Dataset
The 'TinyStories' dataset is a large dataset that was generated using AI story request prompts and responses. It helps facilitate quick fine-tuning of models that were trained with real-world data - such as LLaMA. As this is a large dataset, and this data was generated with the help of AI, the data may have unintentionally been skewed or biased during generation. It is essential to examine the contents of this dataset and whether it is suitable to our applications content-moderation objectives with regard to literacy, mental-health, and creativity.

In [1]:
# importing libraries
from pyspark.sql import SparkSession
from datasets import load_dataset

In [2]:
from pyspark.sql.functions import udf, col, countDistinct
from pyspark.sql.types import StringType, ArrayType, IntegerType
from pyspark.accumulators import AccumulatorParam 

In [38]:
import pandas as pd
from matplotlib import pyplot as plt

## Setting up the Distributed Processing Environment
In this section, the spark distributed cluster is connected using AWS's elastic map reduce service via sagemaker studio.

In [4]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr-serverless connect --application-id 00fra2001bfrlm09 --language python --emr-execution-role-arn arn:aws:iam::597161074694:role/service-role/AmazonEMR-ServiceRole-20250211T131858

Waiting for EMR Serverless application state to become STARTED
Waiting for EMR Serverless application state to become STARTED
Initiating EMR Serverless connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,00fre1dr043tr90a,pyspark,idle,"<a target=""_blank"" href=""https://j-00fre1dr043tr90a.dashboard.emr-serverless.us-east-1.amazonaws.com/?authToken=eDAxjlx1SUIfct_lvwOvVBO9HbV7E9a2pSGGvLw36TSTM67jFf0IInQ_wJ0tI-xyZu1FgtT47emDAkNXTtUhelnsoQj5mgjRTcsh6X1Mv__w4JHHHTpVNouCBqF5ajywPnVar0GvFLgiIob0qNtWXxD2tO7GuLkbacavaviWULqc2hXb4YpCJrk9ZxmJZHgVqzjWkGXA_TKrzXpLq-xmH2yCvgCDLzFL8rXnOitGT4qN6prlbKjlvlM8MfajSMVRyAoINPmKcfklIQVcbTgkVS61bLBHwYsMj7r1GPLbrr6-CCqgmFUNLaAlBxrggzI47qQ-zH3E16BJMCTCsimE_N04eWy6JgG13rAteMqe9gfxcC3Mchb5tAA9PVgAfsAIFqJdsqIcq31zGXjI5f95auAkgXyzDl12CODjZ26qZ8hbYo_03rlLx8AQNzARxqiDMCQ68AVt2kPVnUZWVHxgnse39rZ94Kuq6goymLmlI_c773D11IKHXxMATOiqZUbSxBYxqp-CEoS-BudLb6saMmaFE0RaPIiG1-YD6qCQv0hwXvSDEoRgx2sUxH3h_d4CYAFSlNrNUhBnkRzIl2onLW-ohATvhLq2eEYllYmi4aHb8zskpOBQb-NASFhMB6vvwgT29tOuZQ_gPJepinYCb6r_P_FZobOTLZ86hPlFcPcj10Q58Oa6ifNNNveoTfDhi1SiiO0aD2Be0i5cEsm6UL9ZAxusDNlFUXA7wlAQvU7c_121fOoQbjZs5iXN2yqe9Pv8nUhmDu4SVZYqdU8jwmtuonaOmtsswzUJb6_5vxvvzHQgHC1LfWXnOVHcEv7tqtf7x9c4asOatJ0tgtr2Qw59OlM8j3cVI2JZmTHQODc8HiSIMzplznXOrz_3SkaYT8M4ujsk7AZSLtB2W2FjoDpVXHHkmMKAZ_P9aFtY88X6Rw.eyJraWQiOiJBUUlCQUhnTVJ1MlZyR2FFajR3bTc2dEZqMThGMUsyVFdZbkk4WXpxS0lRamg5YTZHZ0cvOWkyeGZ5dUlDWFhUVCtkdGI2VWZBQUFBb2pDQm53WUpLb1pJaHZjTkFRY0dvSUdSTUlHT0FnRUFNSUdJQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREl5czJCU3AzTVJwTnJwU29BSUJFSUJidi9PVnZSNlZaUGpoY0ZEcE1jNFZBK1MwUEc2SXNUTEpIdzhDRUM3UzRzQnZSRWVGVklnNVVTNTA0clNtZGRhZDR6Nko1UjByb3hzS2tLa2tTNEtOUjZoSTZMYkRWeEMyVVFGcDJ0S2xDZVNldFBTYTM3Nk5BbUR4SlE9PSJ9"">Link","<a target=""_blank"" href=""https://j-00fre1dr043tr90a.dashboard.emr-serverless.us-east-1.amazonaws.com/logs/SPARK_DRIVER/stderr.gz?authToken=eDAxWnokMcjHEo-7JtVa6H54VqoU-S206V34l0L0xaqwqGY4daJU5D5krPBu_PSP0fYdWDi0EsNI_tnSIpDfuk0OwuEJ0Xt6OppQngrT1-fdZnF5PhJXEdghGDLnr_n2muP35ddwTojfke6FgDj-QVxPRhtSckpV4EohiJRBwE8vI2Tbyl54EWzoDM8oBoW_fPUAzJihOY1wpJ_LmuuCOCO1pNiAbGZmQcP8cpvh2FWn2UAEdy68dakE4e_sxRoJnRN5eXPU03m3B3XqZOEs3iicXW946vM4zaniZonJzuoGhuxomWjnRVQwWyOyyVO-IPa94uuh38MiMkZlArej58VFBNTsrgDLnYwRIgFRtVSkVd7S5c1pmiVsJkBhWDctaFigpfGvOgLwGGJZtl1a0B6x9RaicB7W_6j7wYdLWGX5erkA6afm16rogtc2T__9OdPSRl5ObfXQ2HU2GrzGlQXRUMUZEzMu38zbIv6v4Wq1qONU0sJn06D5xeM3dRNsIk02Rm3g9TEjPyz2qBg3x2NQedppJ-Rp_HW004fhER_FL0uXZAS2kalvSCQn-2_GMoQtV4IRZFRH_ZbX2OhtGwXhC6Q_Q700_W-qoDofSM3R-Igej3cmpwl8fikJohL8KMpj1AgSVRxhLafONckv-ne3mEAG7ARQfT1y1OB1GR54TwQ_MDuA_OD1_9FsGAxopP196OJWsC-6UJ9dWqJUjkQrK2e5p6BULHip1MyPAPzxrq6vsr0IgHiXaImcmxCkREO-IglQO7A3jaNiwEZzZx6tyhcNEg-Ql1uow-bExGx6vc4XzWuIOK9JdHmJ5GYrWCdEeREew7ca4HUJ9hT0wO-VkHZp-SaTmG6xpJa8bbymKXc73xfsVzkhO_1CK8Pk6sOUCRa6KmNmIjl95FQibWLoo7F04oUmYHsn0StpyUfjVg.eyJraWQiOiJBUUlCQUhnTVJ1MlZyR2FFajR3bTc2dEZqMThGMUsyVFdZbkk4WXpxS0lRamg5YTZHZ0V0U3l3cEplRWY5OGp5Q1V1aGNQT0NBQUFBb2pDQm53WUpLb1pJaHZjTkFRY0dvSUdSTUlHT0FnRUFNSUdJQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREF3cit1YkczZjQ5RWpCczBnSUJFSUJidlg0WHU3Wm1XUVVNcVFKek4zZWw1MitFdWorVUdVTUJWNURycjZLV09Da1AzYmJQRVlMYS9NL3RMdE9ORnRUR2N1YkNhbTA5VExwSzRITkNvdWlOb0xYRnZEUWkrODlJeEt4QkNlNFd3bER1LzhZdUd3NXZYMmZEalE9PSJ9"">Link",None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [5]:
# connecting to the spark session
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "64g") \
    .appName('spark') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 22:33:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Statistical Metrics
In this section, the basic statistical metrics are calculated to be used for further data analysis and visualization in the following sections.

## Loading the Data

In [6]:
data = load_dataset("skeskinen/TinyStories-GPT4")

In [7]:
train = spark.createDataFrame(data['train'])

## Defining the Spark Functions

In [8]:
class WordCountAccumulator(AccumulatorParam):
    def zero(self, value):
        return {}

    def addInPlace(self, v1, v2):
        # v1 is the accumulator's current state
        # v2 is the new dictionary being added from a partition
        for word, count in v2.items():
            if word is not None and word.strip():
                v1[word] = v1.get(word, 0) + count
        return v1

In [9]:
def normalize_and_count_unique(strings):
    if strings is None:
        return None
    normalized_strings = [s.strip().lower() for s in strings if s is not None and s.strip()]
    return normalized_strings
# the sparkerized user defined function to normalize the words/features for comparison
normalize_count_udf = udf(normalize_and_count_unique, ArrayType(StringType())) # Assuming string type

In [10]:
# applying the accumulator across partitions of the dataset based on col type
def process_features_partition(partition_iterator, col_type):
    # Initialize local counter for this partition
    partition_features = {}
    
    # Process each row in the partition
    for row in partition_iterator:
        features = row["normalized_" + col_type]
        if features is not None:
            # Handle both string and list cases
            words = features.split() if isinstance(features, str) else features
            
            # Count words in this row
            for word in words:
                if word and word.strip():
                    word = word.strip()
                    partition_features[word] = partition_features.get(word, 0) + 1
    
    # Add the partition counts to the accumulator
    if partition_features and col_type == 'features':
        features_accumulator.add(partition_features)
    elif partition_features and col_type == 'words':
        words_accumulator.add(partition_features)
    
    # Return the iterator for the partition
    return iter([1])  # Return dummy value to force evaluation

## Data Prep

### Adding the Normalized Columns For Words/Features
UDF / user defined functions applied on the spark dataset should be added as columns prior to partitioning / converting the dataset.

In [11]:
train = train.withColumn("normalized_features", normalize_count_udf(col("features")))

In [12]:
train = train.withColumn("normalized_words", normalize_count_udf(col("words")))

### Partitioning the Dataset for Mapping the Accumulator

In [13]:
# Ensure proper partitioning
num_partitions = 200  # Adjust based on your cluster size
train = train.repartition(num_partitions)

In [14]:
# Verify the number of partitions
print(f"Number of partitions: {train.rdd.getNumPartitions()}")

25/04/01 22:38:28 WARN TaskSetManager: Stage 0 contains a task of very large size (223671 KiB). The maximum recommended task size is 1000 KiB.
[Stage 0:==============================================>          (13 + 3) / 16]

Number of partitions: 200


[Stage 0:=====================================================>   (15 + 1) / 16]

### Narrative Features

In [15]:
train.columns

['features',
 'prompt',
 'source',
 'story',
 'summary',
 'words',
 'normalized_features',
 'normalized_words']

In [16]:
features_accumulator = spark.sparkContext.accumulator(
    {}, WordCountAccumulator())

In [17]:
# Force evaluation and verify processing
col_type = "features"  # or whatever column name you want to process
total_partitions = train.rdd.mapPartitions(
    lambda partition: process_features_partition(partition, col_type)
).count()

In [18]:
# Get the final word counts
features_accumulator.value

{'dialogue': 1470404,
 'moralvalue': 274152,
 'twist': 539383,
 'foreshadowing': 250789,
 'badending': 250300,
 'conflict': 250696}

In [19]:
feature_occurences = pd.Series(features_accumulator.value)
feature_occurences

dialogue         1470404
moralvalue        274152
twist             539383
foreshadowing     250789
badending         250300
conflict          250696
dtype: int64

### Key Words

In [20]:
words_accumulator = spark.sparkContext.accumulator(
    {}, WordCountAccumulator())

In [21]:
# Force evaluation and verify processing
col_type = "words"  # or whatever column name you want to process
total_partitions = train.rdd.mapPartitions(
    lambda partition: process_features_partition(partition, col_type)
).count()

In [22]:
word_occurrences = pd.Series(words_accumulator.value)
word_occurrences.head()

admire      6971
cane        2546
bald       11245
collect     6956
aunt        2554
dtype: int64

In [23]:
spark.stop()

### Saving the Metrics
As the metrics calculated require a different computation environment for analysis, the metrics are saved to CSV for later use.

In [78]:
feature_occurences.name = 'feature_occurrences'

In [90]:
feature_occurences = feature_occurences.reset_index(drop=False)

In [91]:
feature_occurences.columns = ['feature', 'feature_occurrences']

In [92]:
feature_occurences.to_csv('feature_occurrences.csv', index=False)

In [93]:
word_occurrences.name = 'word_occurrences'

In [94]:
word_occurrences = word_occurrences.reset_index(drop=False)

In [95]:
word_occurrences.columns = ['word', 'word_occurrences']

In [96]:
word_occurrences.to_csv('word_occurrences.csv', index=False)